# Code Inside/Outside of a Function

In [18]:
import time

start_time = time.time()
z = 0
for i in range(0, 10_000_000):
    z += i
print(f"(Global Version) {round(time.time() - start_time, 3)}s taken, result: {z}")


def do():
    z = 0
    for i in range(0, 10_000_000):
        z += i


start_time = time.time()
do()
print(f"(Function Version) {round(time.time() - start_time, 3)}s taken, result: {z}")

(Global Version) 0.568s taken, result: 49999995000000
(Function Version) 0.244s taken, result: 49999995000000


You will notice that the identical code inside a function runs faster than the identical code outside the function (in the global scope).

### Bytecode Analysis

Let's analyze the differences in the disassembly between the two.

In [5]:
import dis

print(dis.dis("""
z = 0
for i in range(0, 10_000_000):
    z += i
"""))

  0           0 RESUME                   0

  2           2 LOAD_CONST               0 (0)
              4 STORE_NAME               0 (z)

  3           6 PUSH_NULL
              8 LOAD_NAME                1 (range)
             10 LOAD_CONST               0 (0)
             12 LOAD_CONST               1 (10000000)
             14 CALL                     2
             22 GET_ITER
        >>   24 FOR_ITER                 7 (to 42)
             28 STORE_NAME               2 (i)

  4          30 LOAD_NAME                0 (z)
             32 LOAD_NAME                2 (i)
             34 BINARY_OP               13 (+=)
             38 STORE_NAME               0 (z)
             40 JUMP_BACKWARD            9 (to 24)

  3     >>   42 END_FOR
             44 RETURN_CONST             2 (None)
None


In [2]:
import dis

print(dis.dis("""
def do():
    z = 0
    for i in range(0, 10_000_000):
        z += i    
"""))

  0           0 RESUME                   0

  2           2 LOAD_CONST               0 (<code object do at 0x107425230, file "<dis>", line 2>)
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (do)
              8 RETURN_CONST             1 (None)

Disassembly of <code object do at 0x107425230, file "<dis>", line 2>:
  2           0 RESUME                   0

  3           2 LOAD_CONST               1 (0)
              4 STORE_FAST               0 (z)

  4           6 LOAD_GLOBAL              1 (NULL + range)
             16 LOAD_CONST               1 (0)
             18 LOAD_CONST               2 (10000000)
             20 CALL                     2
             28 GET_ITER
        >>   30 FOR_ITER                 7 (to 48)
             34 STORE_FAST               1 (i)

  5          36 LOAD_FAST                0 (z)
             38 LOAD_FAST                1 (i)
             40 BINARY_OP               13 (+=)
             44 STORE_FAST            

Notably, please realize that the hot-loop of the code uses `LOAD_NAME` in the global version and `LOAD_FAST` in the function version.

# Import Syntax

In [16]:
import math
import time

start_time = time.time()
z = 0
for i in range(0, 100_000_000):
    z += math.sin(1)
print(f"(math.sin Version) {
      round(time.time() - start_time, 3)}s taken, result: {z}")

from math import sin
start_time = time.time()
z = 0
for i in range(0, 100_000_000):
    z += sin(1)


print(f"(Function Version) {
      round(time.time() - start_time, 3)}s taken, result: {z}")


(math.sin Version) 7.414s taken, result: 84147098.6234422
(Function Version) 8.015s taken, result: 84147098.6234422


## Bytecode Analysis

Let's again analyze the differences in the disassembly here.

In [10]:
import dis

print(dis.dis("""
import math
for i in range(0, 100_000_000):
    z += math.sin(1)"""))


  0           0 RESUME                   0

  2           2 LOAD_CONST               0 (0)
              4 LOAD_CONST               1 (None)
              6 IMPORT_NAME              0 (math)
              8 STORE_NAME               0 (math)

  3          10 PUSH_NULL
             12 LOAD_NAME                1 (range)
             14 LOAD_CONST               0 (0)
             16 LOAD_CONST               2 (100000000)
             18 CALL                     2
             26 GET_ITER
        >>   28 FOR_ITER                23 (to 78)
             32 STORE_NAME               2 (i)

  4          34 LOAD_NAME                3 (z)
             36 PUSH_NULL
             38 LOAD_NAME                0 (math)
             40 LOAD_ATTR                8 (sin)
             60 LOAD_CONST               3 (1)
             62 CALL                     1
             70 BINARY_OP               13 (+=)
             74 STORE_NAME               3 (z)
             76 JUMP_BACKWARD           25 (to 28)

  3

In [11]:
import dis

print(dis.dis("""
from math import sin
for i in range(0, 100_000_000):
    z += sin(1)"""))

  0           0 RESUME                   0

  2           2 LOAD_CONST               0 (0)
              4 LOAD_CONST               1 (('sin',))
              6 IMPORT_NAME              0 (math)
              8 IMPORT_FROM              1 (sin)
             10 STORE_NAME               1 (sin)
             12 POP_TOP

  3          14 PUSH_NULL
             16 LOAD_NAME                2 (range)
             18 LOAD_CONST               0 (0)
             20 LOAD_CONST               2 (100000000)
             22 CALL                     2
             30 GET_ITER
        >>   32 FOR_ITER                13 (to 62)
             36 STORE_NAME               3 (i)

  4          38 LOAD_NAME                4 (z)
             40 PUSH_NULL
             42 LOAD_NAME                1 (sin)
             44 LOAD_CONST               3 (1)
             46 CALL                     1
             54 BINARY_OP               13 (+=)
             58 STORE_NAME               4 (z)
             60 JUMP_BACKWARD

Notice that the `math.sin` calls `LOAD_NAME` on `math` then `LOAD_ATTR` ON `sin`.

# For Loop Semantics

In [17]:
import time

start_time = time.time()
result = [x * 2 for x in range(0, 10_000_000)]
print(f"(List Comprehension) {
      round(time.time() - start_time, 3)}s taken, result: {result[-1]}")

# Example 2: Loop
start_time = time.time()
result = []
for x in range(0, 10_000_000):
    result.append(x * 2)
print(f"(For Loop) {round(time.time() - start_time, 3)
                    }s taken, result: {result[-1]}")

(List Comprehension) 0.292s taken, result: 19999998
(For Loop) 0.522s taken, result: 19999998


## Bytecode Analysis

Let's once again ... you guessed it ... look at the bytecode.

In [14]:
import dis

print(dis.dis("""result = [x * 2 for x in range(0, 10_000_000)]"""))

  0           0 RESUME                   0

  1           2 PUSH_NULL
              4 LOAD_NAME                0 (range)
              6 LOAD_CONST               0 (0)
              8 LOAD_CONST               1 (10000000)
             10 CALL                     2
             18 GET_ITER
             20 LOAD_FAST_AND_CLEAR      0 (x)
             22 SWAP                     2
             24 BUILD_LIST               0
             26 SWAP                     2
        >>   28 FOR_ITER                 7 (to 46)
             32 STORE_FAST               0 (x)
             34 LOAD_FAST                0 (x)
             36 LOAD_CONST               2 (2)
             38 BINARY_OP                5 (*)
             42 LIST_APPEND              2
             44 JUMP_BACKWARD            9 (to 28)
        >>   46 END_FOR
             48 SWAP                     2
             50 STORE_FAST               0 (x)
             52 STORE_NAME               1 (result)
             54 RETURN_CONST       

In [13]:
import dis

print(dis.dis("""
result = []
for x in range(0, 10_000_000):
    result.append(x * 2)"""))

  0           0 RESUME                   0

  2           2 BUILD_LIST               0
              4 STORE_NAME               0 (result)

  3           6 PUSH_NULL
              8 LOAD_NAME                1 (range)
             10 LOAD_CONST               0 (0)
             12 LOAD_CONST               1 (10000000)
             14 CALL                     2
             22 GET_ITER
        >>   24 FOR_ITER                22 (to 72)
             28 STORE_NAME               2 (x)

  4          30 LOAD_NAME                0 (result)
             32 LOAD_ATTR                7 (NULL|self + append)
             52 LOAD_NAME                2 (x)
             54 LOAD_CONST               2 (2)
             56 BINARY_OP                5 (*)
             60 CALL                     1
             68 POP_TOP
             70 JUMP_BACKWARD           24 (to 24)

  3     >>   72 END_FOR
             74 RETURN_CONST             3 (None)
None


The for loop calls `LOAD_ATTR`, then `CALL`, which is less efficient than the implicit `LIST_APPEND`.